# RSI efficiency on Warsaw Stock Exchange based on WIG20

In [1]:
""" append parent dir to sys.path for local imports """
import sys
if '..\\' not in sys.path: sys.path.append('..\\')   

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf
from stocks.stock import Stock
from stocks.stock_index import wig20, mwig40
from analysis.rsi import rsi, rsi_cross_signals

sns.set()
plt.rcParams['figure.figsize'] = [12, 8]

## Time range of interest

In [2]:
start_date = '2019-01-01'
end_date = '2019-12-31'

time_range = pd.date_range(start_date, end_date)

## Get OHLC information, and calculate RSI and signals

In [3]:
stock_data = pd.DataFrame(columns=['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Norm Close', 'Norm Volume', 'RSI', 'Buy', 'Sell'])

all_tickers = wig20
all_tickers.update(mwig40)

for ticker in all_tickers:
    """ create Stock instance """
    ticker_stock = Stock(ticker)
    
    """ get prices from given time range """
    map_ = ticker_stock.ohlc.index.isin(time_range)
    ticker_data = ticker_stock.ohlc[map_]
    
    ticker_data.insert(0, 'Norm Close', ticker_data['Close'] / ticker_data['Close'].max())
    ticker_data.insert(0, 'Norm Volume', ticker_data['Volume'] / ticker_data['Volume'].max())
    
    """ calculate RSI using all available data """
    ticker_data = ticker_data.join(rsi(ticker_stock.ohlc))
    
    """ find Buy and Sell signals """
    ticker_data['Buy'] = rsi_cross_signals(ticker_data, 20, 'onrise').astype(float)
    ticker_data['Sell'] = rsi_cross_signals(ticker_data, 80, 'onrise').astype(float)
    
    """ insert date and ticker """
    ticker_data.insert(0, 'Ticker', ticker)
    ticker_data.insert(0, 'Date', ticker_data.index)
    
    """ collect data in one table """
    stock_data = stock_data.append(ticker_data, ignore_index=True)
    
stock_data.head()

ALR


ValueError: 'Date' is not in list

In [ ]:
stock_data.describe()

In [ ]:
_ = pd.plotting.scatter_matrix(stock_data[['Norm Close', 'Norm Volume', 'RSI', 'Buy', 'Sell']])

In [ ]:
sns.heatmap(stock_data[['Norm Close', 'Norm Volume', 'RSI', 'Buy', 'Sell']].corr())

## Extract close price changes after buy signal

In [ ]:
price_change = pd.DataFrame()
  
for ticker in all_tickers:
    ticker_data = stock_data[stock_data['Ticker']==ticker]
    buy_map = ticker_data['Buy'].astype(bool)
    
    """ for each buy signal """
    for buy in ticker_data[buy_map].index:
        
        """ get close price on the day of the signal and 30 following days (31 days together) """
        price_after_buy_signal = ticker_data.loc[buy:].head(91)['Norm Close']
        
        """ reset index and transpose to a row """
        price_after_buy_signal = price_after_buy_signal.reset_index().drop(['index'], axis=1)
        price_after_buy_signal = price_after_buy_signal.transpose()
        
        """ collect all in one DataFrame """
        price_change = price_change.append(price_after_buy_signal, ignore_index=True)
        
price_change.head()

## Calculate relative price change

In [ ]:
""" calculate price change relative to the price on the day of the signal """
price_change_relative = price_change.div(price_change.loc[:,0], axis=0)
price_change_relative.head()

In [ ]:
price_change_relative.transpose().plot(legend=False)

In [ ]:
price_change_relative.mean(axis=0).plot(legend=False)